In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/1281081789.py:11: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92488,154515,https://forum.effectivealtruism.org/posts/e72Q...,Re-Announcing High Impact Engineers (FKA EA En...,\N,\N,2022-07-31 01:37:22.606614+00,2022-07-31 01:37:22.606614+00,NaN,\N,EAForum,83.0,0.0,\N
92489,154516,https://www.lesswrong.com/posts/f3o9ydY7iPjFF2...,Focusing,\N,\N,2022-07-31 01:37:22.607995+00,2022-07-31 01:37:22.607995+00,NaN,\N,LW,23.0,0.0,\N
92490,154517,https://astralcodexten.substack.com/p/your-boo...,Your Book Review: Viral,\N,\N,2022-07-31 01:37:22.609404+00,2022-07-31 01:37:22.609404+00,NaN,\N,SSC,77.0,0.0,\N
92491,154518,https://thingofthings.substack.com/p/book-revi...,Book Review: History of Psychiatry,\N,\N,2022-07-31 01:37:22.610924+00,2022-07-31 01:37:22.610924+00,NaN,\N,ThingOfThings,48.0,0.0,\N


In [2]:
links['added'].max()

'2022-07-31 01:37:22.612384+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
92300    154317
92301    154318
92432    154454
92433    154456
92434    154457
Name: id, Length: 9599, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
92477,154504,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,\N,2022-07-31 01:37:22.591249+00,2022-07-31 01:37:22.591249+00,NaN,\N,CEA,57.0,0.0,\N


In [5]:
links[links['url'].duplicated()]['id']

92477    154504
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/97
...11/97


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...21/97
...31/97


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...41/97
...51/97


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...61/97
...71/97


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_80426/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...81/97
...91/97
...97/97


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92584,154611,https://mobile.twitter.com/SGRodriques,https://mobile.twitter.com/SGRodriques,\N,\N,2022-07-30 22:52:10.892746,2022-07-30 22:52:10.892757,\N,\N,Custom,54.0,0.0,NaN
92585,154612,https://snowdon.substack.com/p/basic-statistic...,Basic statistics for anti-vaxxers - by Christo...,\N,\N,2022-07-30 22:52:10.903882,2022-07-30 22:52:10.903897,\N,\N,Custom,82.0,0.0,NaN
92586,154613,https://impactfulanimal.substack.com/p/july-ed...,July edition/Impactful Animal Advocacy Newsletter,\N,\N,2022-07-30 22:52:10.915068,2022-07-30 22:52:10.915081,\N,\N,Custom,14.0,0.0,NaN
92587,154614,https://mobile.twitter.com/JgaltTweets/status/...,"(4) JgaltTweets on Twitter: ""@peterwildeford T...",\N,\N,2022-07-30 22:52:10.925477,2022-07-30 22:52:10.925488,\N,\N,Custom,40.0,0.0,NaN


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92584,154611,https://mobile.twitter.com/SGRodriques,https://mobile.twitter.com/SGRodriques,\N,\N,2022-07-30 22:52:10.892746,2022-07-30 22:52:10.892757,\N,\N,Custom,54.0,0.0,NaN
92585,154612,https://snowdon.substack.com/p/basic-statistic...,Basic statistics for anti-vaxxers - by Christo...,\N,\N,2022-07-30 22:52:10.903882,2022-07-30 22:52:10.903897,\N,\N,Custom,82.0,0.0,NaN
92586,154613,https://impactfulanimal.substack.com/p/july-ed...,July edition/Impactful Animal Advocacy Newsletter,\N,\N,2022-07-30 22:52:10.915068,2022-07-30 22:52:10.915081,\N,\N,Custom,14.0,0.0,NaN
92587,154614,https://mobile.twitter.com/JgaltTweets/status/...,"(4) JgaltTweets on Twitter: ""@peterwildeford T...",\N,\N,2022-07-30 22:52:10.925477,2022-07-30 22:52:10.925488,\N,\N,Custom,40.0,0.0,NaN


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92584,154611,https://mobile.twitter.com/SGRodriques,https://mobile.twitter.com/SGRodriques,\N,\N,2022-07-30 22:52:10.892746,2022-07-30 22:52:10.892757,\N,\N,Custom,54,0,NaN
92585,154612,https://snowdon.substack.com/p/basic-statistic...,Basic statistics for anti-vaxxers - by Christo...,\N,\N,2022-07-30 22:52:10.903882,2022-07-30 22:52:10.903897,\N,\N,Custom,82,0,NaN
92586,154613,https://impactfulanimal.substack.com/p/july-ed...,July edition/Impactful Animal Advocacy Newsletter,\N,\N,2022-07-30 22:52:10.915068,2022-07-30 22:52:10.915081,\N,\N,Custom,14,0,NaN
92587,154614,https://mobile.twitter.com/JgaltTweets/status/...,"(4) JgaltTweets on Twitter: ""@peterwildeford T...",\N,\N,2022-07-30 22:52:10.925477,2022-07-30 22:52:10.925488,\N,\N,Custom,40,0,NaN


In [10]:
links.to_csv('data/export.csv', index=False)